In [1]:
import os
os.chdir('/home/guest/d11922022/github/openai-gym')
import gymnasium as gym
from gym.wrappers import RecordVideo
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib created a temporary cache directory at /tmp/matplotlib-39y19imy because the default path (/home/guest/d11922022/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import wandb
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
# https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW12/HW12_EN.pdf
# https://github.com/john-hu/rl
# https://medium.com/no-sliver-bullet/%E5%BC%B7%E5%8C%96%E5%AD%B8%E7%BF%92-reinforcement-learning-lunar-lander-v2-1291d48b71c3
# https://github.com/nikhilbarhate99/Actor-Critic-PyTorch/tree/master/preTrained
"""
Rewards
Reward for moving from the top of the screen to the landing pad and coming to rest is about 100-140 
points. If the lander moves away from the landing pad, it loses reward. If the lander crashes, it 
receives an additional -100 points. If it comes to rest, it receives an additional +100 points. Each 
leg with ground contact is +10 points. Firing the main engine is -0.3 points each frame. Firing the 
side engine is -0.03 points each frame. Solved is 200 points.
"""

from utils import plot_rewards


In [3]:
from scripts.train import trainer
from agents import (
    PolicyGradientNetwork,
    PolicyGradientAgent
) 

input_size, hidden_sizes, output_size = 8, [10, 10], 4
network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
agent = PolicyGradientAgent(network)

training_config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "discounted_future_reward",
    "learning_rate": 1e-3,
    "gamma": 0.99,
    "batch": 500,
    "episode": 5,
    "model_path": "",
    "model_name": "",
    "greedy": False,
    "info": "network 8x10x10x4"
}

def discounted_future_reward(seq_rewards, rewards, gamma):
    n = len(seq_rewards)

    for i in range(2, n+1):
        seq_rewards[-i] += gamma * (seq_rewards[-i+1])
    rewards[-len(seq_rewards):] = seq_rewards
    return rewards

luna_lander_trainer = trainer(agent, training_config, seed=1)
avg_total_reward, avg_final_reward, avg_episode_steps = luna_lander_trainer.train(discounted_future_reward)
plot_rewards(avg_total_rewards, avg_final_rewards)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: d11922022. Use `wandb login --relogin` to force relogin


  0%|          | 0/500 [00:00<?, ?it/s]

NameError: name 'avg_total_rewards' is not defined

In [ ]:
luna_lander_trainer.config['greedy'] = False
total_rewards, final_rewards, episode_steps = luna_lander_trainer.test(discounted_future_reward)
plot_rewards(total_rewards, final_rewards)

In [ ]:
# 參考
# https://github.com/gabrielgarza/openai-gym-policy-gradient/blob/master/policy_gradient.py
# https://github.com/sudharsan13296/Hands-On-Reinforcement-Learning-With-Python/blob/master/11.%20Policy%20Gradients%20and%20Optimization/11.2%20Lunar%20Lander%20Using%20Policy%20Gradients.ipynb
# https://github.com/PacktPublishing/Hands-on-Reinforcement-Learning-with-PyTorch/blob/master/Section%204/4.3%20Policy%20Gradients%20REINFORCE%20Baseline.ipynb


In [ ]:
SEED = 543

def fix(env, seed):
    """
    Fix random seed for reproducibility.

    Args:
        env: Environment object.
        seed (int): Seed value for random number generation.

    Notes:
        This function fixes the random seed for various libraries and modules
        to ensure reproducibility of results in environments with randomness.

    Deprecated:
        - env.seed(seed): Deprecated method to set seed for environment.
        - torch.set_deterministic(True): Deprecated method to set PyTorch 
          deterministic mode.

    """
    # -deprecated- env.seed(seed)
    env.reset(seed=seed)
    env.action_space.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed = seed
    # -deprecated- torch.set_deterministic(True)
    torch.use_deterministic_algorithms = True
    torch.are_deterministic_algorithms_enabled = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


In [ ]:
def run_example():
    """
    Run an example of LunarLander-v2 environment.

    Notes:
        This function demonstrates running an example of the LunarLander-v2
        environment using Gym. It fixes the random seed for reproducibility,
        renders the environment, and displays the frames.

    """
    env = gym.make('LunarLander-v2', render_mode='rgb_array')
    fix(env, SEED)
    env.reset()
    img = plt.imshow(env.render()) # only call this once
    total_reward = 0
    for _ in range(100):
        img.set_data(env.render()) # just update the data
        display.display(plt.gcf())
        display.clear_output(wait=True)
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        print(f'\r{reward:.4f}, total_reward:{total_reward:.4f}, terminated:{terminated}')

run_example()

In [ ]:
# env.observation_space
# env.action_space
# initial_state, initial_info = env.reset(seed=seed)
# random_action = env.action_space.sample()
# observation, reward, terminated, truncated, info = env.step(random_action)

In [ ]:
# import os
# import gymnasium as gym
# env = gym.make("LunarLander-v2", render_mode="rgb_array")
# trigger = lambda t: t % 10 == 0
# env = RecordVideo(env, video_folder="./video/video1", episode_trigger=trigger, disable_logger=True)
# for i in range(50):
#     termination, truncation = False, False
#     _ = env.reset(seed=123)
#     while not (termination or truncation):
#         obs, rew, termination, truncation, info = env.step(env.action_space.sample())

# env.close()
# len(os.listdir("./video/video1"))

# Random Agent

In [ ]:
def demo_run(env, agent, seed=SEED, num_episode=1, record_video=False, video_folder = './video/tmp', greedy=False):
    agent.network.eval()
    
    if not record_video:
        img = plt.imshow(env.render())
    
    for episode in range(num_episode):
        if not record_video:
            env_ = env
        else:
            trigger = lambda t: t%10 == 0
            action_mode = 'greedy' if greedy else 'sample'
            env_ = RecordVideo(env, 
                               video_folder=video_folder+f'-{action_mode}-ep{episode}', 
                               episode_trigger=trigger)
            
        state, info = env_.reset(seed=seed)
        terminated, truncated = False, False
        total_reward = 0
        while not (terminated or truncated):
            action, log_prob = agent.sample(state, greedy)
            next_state, reward, terminated, truncated, info = env_.step(action)
            state = next_state
            total_reward += reward
            print(f'reward: {reward}, total_reward:{total_reward}, terminated:{terminated}, truncated:{truncated}')

            if not record_video:
                img.set_data(env.render())
                display.display(plt.gcf())
                display.clear_output(wait=True)
        print(f'reward: {reward}, total_reward:{total_reward}, terminated:{terminated}, truncated:{truncated}')

# Policy Gradient

In [ ]:
class PolicyGradientNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 10)
        self.fc2 = nn.Linear(10,10)
        self.fc3 = nn.Linear(10, 4)

    def forward(self, state):
        hid = torch.relu(self.fc1(state))
        hid = torch.relu(self.fc2(hid))
        return F.softmax(self.fc3(hid), dim=-1)
    

import torch
import torch.nn as nn
import torch.nn.functional as F

class PolicyGradientNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = nn.ModuleList()
        
        # Add input layer
        self.hidden_layers.append(nn.Linear(input_size, hidden_sizes[0]))
        
        # Add hidden layers
        for i in range(len(hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
        
        # Add output layer
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, state):
        # Forward pass through each hidden layer
        for layer in self.hidden_layers:
            state = F.relu(layer(state))
        
        # Output layer
        return F.softmax(self.output_layer(state), dim=-1)

# Example usage:
input_size = 8
hidden_sizes = [10, 10]  # You can adjust this list to add more hidden layers or change sizes
output_size = 4

# Initialize the network
network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)

# # Example forward pass
# state = torch.randn(1, input_size)  # Example input
# output = model(state)
# print(output)


In [ ]:
class ActorCriticNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16,16)

        # Actor network
        self.actor_fc3 = nn.Linear(16, 4)

        # Critic network
        self.critic_fc3 = nn.Linear(16, 1)

    def forward(self, state):
        hid = torch.relu(self.fc1(state))
        hid = torch.relu(self.fc2(hid))

        # Actor branch
        actor_hid = torch.relu(hid)
        actor_out = F.softmax(self.actor_fc3(actor_hid), dim=-1)

        # Critic branch
        critic_hid = torch.relu(hid)
        critic_out = self.critic_fc3(critic_hid)

        return actor_out, critic_out

Then, we need to build a simple agent. The agent will acts according to the output of the policy network above. There are a few things can be done by agent:

* `learn()`：update the policy network from log probabilities and rewards.
* `sample()`：After receiving observation from the environment, utilize policy network to tell which action to take. The return values of this function includes action and log probabilities.

In [ ]:
class PolicyGradientAgent():
    def __init__(self, network, lr=1e-3, epochs=500):
        self.lr = lr
        self.epochs = epochs
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=1e-3)
        # self.lr_scheduler = CosineAnnealingLR(self.optimizer, self.epochs, eta_min=1e-6)

    def forward(self, state):
        return self.network(state)

    def learn(self, log_probs, rewards):
        loss = (-log_probs * rewards).sum() # You don't need to revise this to pass simple baseline (but you can)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        # self.lr_scheduler.step()
        
    def sample(self, state, greedy=False):
        action_prob = self.network(torch.FloatTensor(np.array(state)))
        action_dist = Categorical(action_prob)
        if not greedy:
            action = action_dist.sample()
        else:
            action = action_prob.argmax()
        log_prob = action_dist.log_prob(action)
        return action.item(), log_prob

    def save(self, PATH): # You should not revise this
        Agent_Dict = {
            "network" : self.network.state_dict(),
            "optimizer" : self.optimizer.state_dict()
        }
        torch.save(Agent_Dict, PATH)

    def load(self, PATH): # You should not revise this
        checkpoint = torch.load(PATH)
        self.network.load_state_dict(checkpoint["network"])
        #如果要儲存過程或是中斷訓練後想繼續可以用喔
        self.optimizer.load_state_dict(checkpoint["optimizer"])


In [ ]:
class ActorCriticAgent():
    def __init__(self, network, lr=1e-3, epochs=500):
        self.lr = lr
        self.epochs = epochs
        self.network = network
        self.optimizer_actor = optim.SGD(self.network.parameters(), lr=lr)
        self.optimizer_critic = optim.SGD(self.network.parameters(), lr=lr)  # Use the same learning rate for simplicity
        self.lr_scheduler_actor = CosineAnnealingLR(self.optimizer_actor, self.epochs, eta_min=1e-6)
        self.lr_scheduler_critic = CosineAnnealingLR(self.optimizer_critic, self.epochs, eta_min=1e-6)

    def forward(self, state):
        return self.network(state)

    def actor_learn(self, log_probs, rewards):
        loss = (-log_probs * rewards).sum()
        self.optimizer_actor.zero_grad()
        loss.backward()
        self.optimizer_actor.step()
        self.lr_scheduler_actor.step()

    def critic_learn(self, values, rewards):
        loss = F.mse_loss(values, rewards)
        self.optimizer_critic.zero_grad()
        loss.backward()
        self.optimizer_critic.step()
        self.lr_scheduler_critic.step()

    def sample(self, state, greedy=False):
        action_prob, value = self.network(torch.FloatTensor(np.array(state)))
        
        # actor branch
        action_dist = Categorical(action_prob)
        if not greedy:
            action = action_dist.sample()
        else:
            action = action_prob.argmax()
        log_prob = action_dist.log_prob(action)

        return action.item(), log_prob, value
 

In [ ]:
# # build a network and agent to start training
# network = PolicyGradientNetwork()
# agent = PolicyGradientAgent(network)

# Training Agent

In [ ]:
# import wandb
# import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="lunalander",
    
#     # track hyperparameters and run metadata
#     config={
#     "Agent": "PolicyGradientAgent",
#     "reward": "normalized(step_reward)*step_log_prob",
#     "learning_rate": 1e-3,
#     "epochs": 500,
#     }
# )

In [ ]:
# # Example usage:
input_size = 8
hidden_sizes = [10, 10]  # You can adjust this list to add more hidden layers or change sizes
output_size = 4

# Initialize the network
# network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)

from utils import fix
# network = PolicyGradientNetwork()
network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
agent = PolicyGradientAgent(network)

In [ ]:
from utils import fix
network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

# network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
# agent = PolicyGradientAgent(network)

env = gym.make('LunarLander-v2', render_mode='rgb_array')
fix(env, 1)

agent.network.train()  # 訓練前，先確保 network 處在 training 模式
EPISODE_PER_BATCH = 5  # 每蒐集 5 個 episodes 更新一次 agent
NUM_BATCH =500        # 總共更新 500 次

avg_total_rewards, avg_final_rewards = [], []

gamma = 0.992
prg_bar = tqdm(range(NUM_BATCH))
for batch in prg_bar:

    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []

    # 蒐集訓練資料
    for episode in range(EPISODE_PER_BATCH):
        
        state, info = env.reset()
        total_reward, total_step = 0, 0
        seq_rewards = []
        while True:

            action, log_prob = agent.sample(state) # at, log(at|st)
            next_state, reward, terminated, truncated, info = env.step(action)

            log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
            seq_rewards.append(reward)
            state = next_state
            total_reward += reward
            total_step += 1
            rewards.append(reward) #改這裡
            # ! 重要 ！
            # 現在的reward 的implementation 為每個時刻的瞬時reward, 給定action_list : a1, a2, a3 ......
            #                                                       reward :     r1, r2 ,r3 ......
            # medium：將reward調整成accumulative decaying reward, 給定action_list : a1,                         a2,                           a3 ......
            #                                                       reward :     r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,r3+0.99*r4+0.99^2*r5+ ......
            # boss : implement DQN
            
            if terminated or truncated:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                
                n = len(seq_rewards)
                # for i in range(n-1, -1, -1):
                #     rewards[i-n] = sum([r*gamma**j for j, r in enumerate(seq_rewards[i:], 0)])

                for i in range(2, n+1):
                    seq_rewards[-i] += gamma * (seq_rewards[-i+1])
                rewards[-len(seq_rewards):] = seq_rewards
                break
        agent.save(f'./models/policy_gradient@{episode}.pt')
    

    

    # print(f"rewards looks like ", np.shape(rewards))  
    # print(f"log_probs looks like ", np.shape([_.item() for _ in log_probs]))    
    # 紀錄訓練過程
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # 更新網路
    # rewards = np.concatenate(rewards, axis=0)
    rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # 將 reward 正規標準化
    agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
    # print("logs prob looks like ", torch.stack(log_probs).size())
    # print("torch.from_numpy(rewards) looks like ", torch.from_numpy( np.array(rewards) ).size())

plot_rewards(total_rewards)
plot_rewards(avg_total_rewards, avg_final_rewards)

In [ ]:
demo_run(env, agent, seed=None, num_episode=5, record_video=True, video_folder = './video/tmp', greedy=False)

In [ ]:
demo_run(env, agent, seed=None, num_episode=5, record_video=True, video_folder = './video/tmp', greedy=True)

In [ ]:
# Initialize the network
from utils import fix

input_size = 8
hidden_sizes = [10, 10]  # You can adjust this list to add more hidden layers or change sizes
output_size = 4
network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
agent = PolicyGradientAgent(network)

# network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
# agent = PolicyGradientAgent(network)

env = gym.make('LunarLander-v2', render_mode='rgb_array')
fix(env, 1)

agent.network.train()  # 訓練前，先確保 network 處在 training 模式
EPISODE_PER_BATCH = 5  # 每蒐集 5 個 episodes 更新一次 agent
NUM_BATCH =500        # 總共更新 500 次

avg_total_rewards, avg_final_rewards = [], []

gamma = 0.992
prg_bar = tqdm(range(NUM_BATCH))
for batch in prg_bar:

    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []

    # 蒐集訓練資料
    for episode in range(EPISODE_PER_BATCH):
        
        state, info = env.reset()
        total_reward, total_step = 0, 0
        seq_rewards = []
        while True:

            action, log_prob = agent.sample(state) # at, log(at|st)
            next_state, reward, terminated, truncated, info = env.step(action)

            log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
            seq_rewards.append(reward)
            state = next_state
            total_reward += reward
            total_step += 1
            rewards.append(reward) #改這裡
            # ! 重要 ！
            # 現在的reward 的implementation 為每個時刻的瞬時reward, 給定action_list : a1, a2, a3 ......
            #                                                       reward :     r1, r2 ,r3 ......
            # medium：將reward調整成accumulative decaying reward, 給定action_list : a1,                         a2,                           a3 ......
            #                                                       reward :     r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,r3+0.99*r4+0.99^2*r5+ ......
            # boss : implement DQN
            
            if terminated or truncated:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                
                n = len(seq_rewards)
                # for i in range(n-1, -1, -1):
                #     rewards[i-n] = sum([r*gamma**j for j, r in enumerate(seq_rewards[i:], 0)])

                for i in range(2, n+1):
                    seq_rewards[-i] += gamma * (seq_rewards[-i+1])
                rewards[-len(seq_rewards):] = seq_rewards
                break
        agent.save(f'./models/policy_gradient2@{episode}.pt')
    

    

    # print(f"rewards looks like ", np.shape(rewards))  
    # print(f"log_probs looks like ", np.shape([_.item() for _ in log_probs]))    
    # 紀錄訓練過程
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # 更新網路
    # rewards = np.concatenate(rewards, axis=0)
    rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # 將 reward 正規標準化
    agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
    # print("logs prob looks like ", torch.stack(log_probs).size())
    # print("torch.from_numpy(rewards) looks like ", torch.from_numpy( np.array(rewards) ).size())

plot_rewards(total_rewards)
plot_rewards(avg_total_rewards, avg_final_rewards)

In [ ]:
demo_run(env, agent, seed=None, num_episode=5, record_video=True, video_folder = './video/tmp2', greedy=False)

In [ ]:
network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)


config = {"reward":"MC"}

agent.network.train()  # Switch network into training mode 
EPISODE_PER_BATCH = 5  # update the  agent every 5 episode
NUM_BATCH = 10        # totally update the agent for 400 time
gamma = 0.9

avg_total_rewards, avg_final_rewards = [], []


NUM_BATCH=1
EPISODE_PER_BATCH = 1
prg_bar = tqdm(range(NUM_BATCH))

    
for batch in range(NUM_BATCH):#prg_bar:
    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []
    total_steps = []
    
    # collect trajectory
    for episode in range(EPISODE_PER_BATCH):
        
        state, info = env.reset()
        img = plt.imshow(env.render())
        total_reward, total_step = 0, 0
        terminated, truncated = False, False
        while not(terminated or truncated):
        # while not (terminated or truncated):
            print(f'\r{reward:.4f}, total_reward:{total_reward:.4f}, terminated:{terminated}')
            action, log_prob = agent.sample(state) # at, log(at|st)
            next_state, reward, terminated, truncated, info = env.step(action)

            # render
            img.set_data(env.render()) # just update the data
            display.display(plt.gcf())
            display.clear_output(wait=True)     
            # render       

            log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
            state = next_state
            total_reward += reward
            match config["reward"]:
                case "normalized_step_reward":
                    rewards.append(reward) # change here
                case "MC":
                    rewards.append(reward*(gamma**total_step))
    
            total_step += 1

            # ! IMPORTANT !
            # Current reward implementation: immediate reward,  given action_list : a1, a2, a3 ......
            #                                                         rewards :     r1, r2 ,r3 ......
            # medium：change "rewards" to accumulative decaying reward, given action_list : a1,                           a2,                           a3, ......
            #                                                           rewards :           r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,  r3+0.99*r4+0.99^2*r5+ ......
            # boss : implement Actor-Critic

            # if terminated or truncated:
            #    final_rewards.append(reward)
            #    total_rewards.append(total_reward)
            #    break

        else:
            final_rewards.append(reward)
            total_rewards.append(total_reward)
            total_steps.append(total_step)
            state, info = env.reset()
            #img.set_data(env.render()) # just update the data
            #display.display(plt.gcf())
            #display.clear_output(wait=True)     
        
    
    # update agent
    mean_rewards = np.mean(rewards)
    std_rewards = np.std(rewards)

    match config["reward"]:
        case "normalized_step_reward":
            rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
        case "MC":
            rewards_ = []
            start = [sum(total_steps[:i]) for i in range(len(total_steps))]
            end =  [sum(total_steps[:i]) for i in range(1, len(total_steps)+1)]

            for s, e in zip(start, end):
                rewards_ = rewards_ + [ sum(rewards[s:e][i:])/(gamma**i) for i in range(len(rewards[s:e])) ]
            
            #rewards = rewards_
            #rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)
    

    #print(f"rewards looks like ", np.shape(rewards))  
    #print(f"log_probs looks like ", np.shape(log_probs))     
    # record training process
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # update agent
    # rewards = np.concatenate(rewards, axis=0)
    mean_rewards = np.mean(rewards)
    std_rewards = np.std(rewards)
    # rewards = (np.array(rewards) - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
    #rewards = np.array(np.sum(rewards))
    #rewards = np.array([sum(rewards[i:])/(gamma**i) for i in range(len(rewards))])
    agent.learn(torch.stack(log_probs), torch.from_numpy(np.array(rewards_)))
    # print("logs prob looks like ", torch.stack(log_probs).size())
    # print("torch.from_numpy(rewards) looks like ", torch.from_numpy( np.array(rewards_) ).size())
    
    # wandb.log({"mean rewards":mean_rewards, 
    #            "std reward":std_rewards, 
    #            "avg_total_reward": avg_total_reward, 
    #            "avg_final_reward": avg_final_reward, 
    #            "avg_episode_steps":len(rewards)/EPISODE_PER_BATCH}
    # )

In [ ]:
EPISODE_PER_BATCH = 5  # update the  agent every 5 episode
NUM_BATCH = 500        # totally update the agent for 400 time

def agent_run(
    env, 
    agent, 
    config, 
    episode_per_batch=EPISODE_PER_BATCH, 
    num_batch=NUM_BATCH, 
    gamma=0.99,
    step_punish = -0.15,
    wandb_log = True,
    verbose = False
):

    if wandb_log:
        wandb.init(
            # set the wandb project where this run will be logged
            project = config['project'], 
            # track hyperparameters and run metadata
            config = config
        )

    match config['phase']:
        case 'train':
            agent.network.train()  # Switch network into training mode 
        case 'eval':
            agent.network.eval()  # Switch network into eval mode 
    
    num_batch, episode_per_batch = config['num_batch'], config['episode_per_batch']
    avg_total_rewards, avg_final_rewards = [], []

    prg_bar = tqdm(range(num_batch))
    for batch in prg_bar:

        log_probs, rewards = [], []
        total_rewards, final_rewards = [], []
        total_steps = []
        # collect trajectory
        for episode in range(episode_per_batch):
            
            state, info = env.reset()
            total_reward, total_step = 0, 0
            terminated, truncated = False, False
            
            while not(terminated or truncated):
            # while not (terminated or truncated):
                action, log_prob = agent.sample(state) # at, log(at|st)
                next_state, reward, terminated, truncated, info = env.step(action)
                reward += step_punish
                # if reward == -100:
                #     reward = -10
                

                log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
                state = next_state
                total_reward += reward
                match config["reward"]:
                    case "normalized_step_reward":
                        rewards.append(reward) # change here
                    case "MC":
                        rewards.append(reward*(gamma**total_step))
                
                total_step += 1
                # ! IMPORTANT !
                # Current reward implementation: immediate reward,  given action_list : a1, a2, a3 ......
                #                                                         rewards :     r1, r2 ,r3 ......
                # medium：change "rewards" to accumulative decaying reward, given action_list : a1,                           a2,                           a3, ......
                #                                                           rewards :           r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,  r3+0.99*r4+0.99^2*r5+ ......
                # boss : implement Actor-Critic

                # if terminated or truncated:
                #    final_rewards.append(reward)
                #    total_rewards.append(total_reward)
                #    break

            else:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                total_steps.append(total_step)
                
            

        # record training process
        avg_total_reward = sum(total_rewards) / len(total_rewards)
        avg_final_reward = sum(final_rewards) / len(final_rewards)
        avg_total_rewards.append(avg_total_reward)
        avg_final_rewards.append(avg_final_reward)
        avg_episode_steps = sum(total_steps)/len(total_steps)
        prg_bar.set_description(f"\r Total reward: {avg_total_reward: 4.1f}, Final reward: {avg_final_reward: 4.1f}")

        # update agent
        mean_rewards = np.mean(rewards)
        std_rewards = np.std(rewards)

        match config["reward"]:
            case "normalized_step_reward":
                rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
            case "MC":
                rewards_ = []
                start = [sum(total_steps[:i]) for i in range(len(total_steps))]
                end =  [sum(total_steps[:i]) for i in range(1, len(total_steps)+1)]

                for s, e in zip(start, end):
                    rewards_ = rewards_ + [ sum(rewards[s:e][i:])/(gamma**i) for i in range(len(rewards[s:e])) ]
                
                rewards = rewards_
                rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)
        
                #rewards = np.array([sum(rewards[i:])/(gamma**i) for i in range(len(rewards))])
                #rewards = np.array(np.sum(rewards))

        agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
        if verbose:
            print("logs prob looks like ", torch.stack(log_probs).size())
            print("torch.from_numpy(rewards) looks like ", torch.from_numpy(rewards).size())

        if wandb_log:
            wandb.log({"mean rewards":mean_rewards, 
                    "std reward":std_rewards, 
                    "avg_total_reward": avg_total_reward, 
                    "avg_final_reward": avg_final_reward, 
                    "avg_episode_steps":avg_episode_steps}
            )

    match config['phase']:
        case 'train':
            torch.save(agent.network.state_dict(), config['output_dir'])


In [ ]:
EPISODE_PER_BATCH = 5  # update the  agent every 5 episode
NUM_BATCH = 1        # totally update the agent for 400 time

# def agent_run(
#     env, 
#     agent, 
#     config, 
#     episode_per_batch=EPISODE_PER_BATCH, 
#     num_batch=NUM_BATCH, 
#     gamma=0.99,
#     wandb_log = True,
#     verbose = False
# ):
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 1,
    "episode_per_batch": 5,
    "output_dir": "./models/policy_gradient_normalized_step_reward"
}
episode_per_batch=EPISODE_PER_BATCH 
num_batch=NUM_BATCH
gamma=0.99
wandb_log = True
verbose = False

if True:

    if wandb_log:
        wandb.init(
            # set the wandb project where this run will be logged
            project = config['project'], 
            # track hyperparameters and run metadata
            config = config
        )

    match config['phase']:
        case 'train':
            agent.network.train()  # Switch network into training mode 
        case 'eval':
            agent.network.eval()  # Switch network into eval mode 
    
    num_batch, episode_per_batch = config['num_batch'], config['episode_per_batch']
    avg_total_rewards, avg_final_rewards = [], []

    prg_bar = tqdm(range(num_batch))
    for batch in prg_bar:

        log_probs, rewards = [], []
        total_rewards, final_rewards = [], []
        total_steps = []
        original_rewards = []
        # collect trajectory
        for episode in range(episode_per_batch):
            
            state, info = env.reset()
            total_reward, total_step = 0, 0
            terminated, truncated = False, False
            
            while not(terminated or truncated):
            # while not (terminated or truncated):
                action, log_prob = agent.sample(state) # at, log(at|st)
                next_state, reward, terminated, truncated, info = env.step(action)
                

                log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
                state = next_state
                total_reward += reward
                match config["reward"]:
                    case "normalized_step_reward":
                        rewards.append(reward) # change here
                    case "MC":
                        rewards.append(reward*(gamma**total_step))
                        original_rewards.append(reward)
                
                total_step += 1
                # ! IMPORTANT !
                # Current reward implementation: immediate reward,  given action_list : a1, a2, a3 ......
                #                                                         rewards :     r1, r2 ,r3 ......
                # medium：change "rewards" to accumulative decaying reward, given action_list : a1,                           a2,                           a3, ......
                #                                                           rewards :           r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,  r3+0.99*r4+0.99^2*r5+ ......
                # boss : implement Actor-Critic

                # if terminated or truncated:
                #    final_rewards.append(reward)
                #    total_rewards.append(total_reward)
                #    break

            else:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                total_steps.append(total_step)
                
            

        # record training process
        avg_total_reward = sum(total_rewards) / len(total_rewards)
        avg_final_reward = sum(final_rewards) / len(final_rewards)
        avg_total_rewards.append(avg_total_reward)
        avg_final_rewards.append(avg_final_reward)
        avg_episode_steps = sum(total_steps)/len(total_steps)
        prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

        # update agent
        mean_rewards = np.mean(rewards)
        std_rewards = np.std(rewards)

        match config["reward"]:
            case "normalized_step_reward":
                rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
            case "MC":
                rewards_ = []
                start = [sum(total_steps[:i]) for i in range(len(total_steps))]
                end =  [sum(total_steps[:i]) for i in range(1, len(total_steps)+1)]

                for s, e in zip(start, end):
                    rewards_ = rewards_ + [ sum(rewards[s:e][i:])/(gamma**i) for i in range(len(rewards[s:e])) ]
                
                rewards = np.array(rewards_)
                #rewards = rewards_
                #rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)


        agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
        if verbose:
            print("logs prob looks like ", torch.stack(log_probs).size())
            print("torch.from_numpy(rewards) looks like ", torch.from_numpy(rewards).size())

        if wandb_log:
            wandb.log({"mean rewards":mean_rewards, 
                    "std reward":std_rewards, 
                    "avg_total_reward": avg_total_reward, 
                    "avg_final_reward": avg_final_reward, 
                    "avg_episode_steps":avg_episode_steps}
            )

    match config['phase']:
        case 'train':
            torch.save(agent.network.state_dict(), config['output_dir'])


In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "normalized_step_reward",
    "learning_rate": 1e-3,
    "num_batch": 500,
    "episode_per_batch": 5,
    "output_dir": "./models/policy_gradient_normalized_step_reward"
}

agent_run(env, agent, config=config)


In [ ]:
network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_normalized_step_reward", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_normalized_step_reward", greedy=False)

In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 500,
    "episode_per_batch": 5,
    "output_dir": "./models/policy_gradient_MC_gamma_0.7"
}

agent_run(env, agent, config=config, gamma=0.7)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.7_advantage_step_punish", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.7_advantage_step_punish", greedy=False)


In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 500,
    "episode_per_batch": 1,
    "output_dir": "./models/policy_gradient_MC_gamma_0.90"
}

agent_run(env, agent, config=config, gamma=0.90)


network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.90", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.90", greedy=False)


In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 500,
    "episode_per_batch": 5,
    "output_dir": "./models/policy_gradient_MC_gamma_0.90_episode_per_batch_5_normalized"
}

agent_run(env, agent, config=config, gamma=0.90)


network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_090_episode_per_batch_5_normalized", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_090_episode_per_batch_5_normalized", greedy=False)


In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)

config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 10000,
    "episode_per_batch": 1,
    "output_dir": "./models/policy_gradient_MC_gamma_0.90"
}

agent_run(env, agent, config=config, gamma=0.8)


network = PolicyGradientNetwork()
agent = PolicyGradientAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.90_batch_10000", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/policy_gradient_MC_gamma_0.90_batch_10000", greedy=False)


# Actor Critic

In [ ]:
def demo_run(env, agent, seed=seed, num_episode=1, record_video=False, video_folder = './video/tmp', greedy=False):
    agent.network.eval()
    
    if not record_video:
        img = plt.imshow(env.render())
    
    for episode in range(num_episode):
        if not record_video:
            env_ = env
        else:
            trigger = lambda t: t%10 == 0
            action_mode = 'greedy' if greedy else 'sample'
            env_ = RecordVideo(env, video_folder=video_folder+f'-{action_mode}-{episode}', episode_trigger=trigger)
            
        state, info = env_.reset(seed=seed)
        terminated, truncated = False, False
        while not (terminated or truncated):
            action, log_prob, value = agent.sample(state, greedy)
            next_state, reward, terminated, truncated, info = env_.step(action)
            state = next_state

            if not record_video:
                img.set_data(env.render())
                display.display(plt.gcf())
                display.clear_output(wait=True)

In [ ]:
NUM_BATCH=500
EPISODE_PER_BATCH = 5

network = ActorCriticNetwork()
agent = ActorCriticAgent(network, epochs=NUM_BATCH//2)

config={
    "agent": "ActorCriticAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 500,
    "episode_per_batch": 5,
    "output_dir": "./models/policy_gradient_actor_critcs"
}
wandb.init(
    # set the wandb project where this run will be logged
    project = 'lunalander', 
    # track hyperparameters and run metadata
    config = config
)

config = {"reward":"MC"}

agent.network.train()  # Switch network into training mode 
EPISODE_PER_BATCH = 5  # update the  agent every 5 episode
NUM_BATCH = 500       # totally update the agent for 400 time
gamma = 0.9

avg_total_rewards, avg_final_rewards = [], []



prg_bar = tqdm(range(NUM_BATCH))

    
for b, batch in enumerate(prg_bar):
    log_probs, rewards, values = [], [], []
    total_rewards, final_rewards = [], []
    total_steps = []
    advantages = []
    
    # collect trajectory
    for episode in range(EPISODE_PER_BATCH):
        
        state, info = env.reset()
        #img = plt.imshow(env.render())
        total_reward, total_step = 0, 0
        terminated, truncated = False, False
        while not(terminated or truncated):
        # while not (terminated or truncated):
            with torch.no_grad():
                action, log_prob, value = agent.sample(state) # at, log(at|st)
            next_state, reward, terminated, truncated, info = env.step(action)

            with torch.no_grad():
                _, _, next_value = agent.sample(next_state)

            log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
            state = next_state
            total_reward += reward
            match config["reward"]:
                case "normalized_step_reward":
                    rewards.append(reward) # change here
                case "MC":
                    rewards.append(reward*(gamma**total_step))
                    values.append(value)
                    advantages.append(reward + next_value - value)
    
            total_step += 1

            # ! IMPORTANT !
            # Current reward implementation: immediate reward,  given action_list : a1, a2, a3 ......
            #                                                         rewards :     r1, r2 ,r3 ......
            # medium：change "rewards" to accumulative decaying reward, given action_list : a1,                           a2,                           a3, ......
            #                                                           rewards :           r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,  r3+0.99*r4+0.99^2*r5+ ......
            # boss : implement Actor-Critic

            # if terminated or truncated:
            #    final_rewards.append(reward)
            #    total_rewards.append(total_reward)
            #    break

        else:
            final_rewards.append(reward)
            total_rewards.append(total_reward)
            total_steps.append(total_step)
            state, info = env.reset()
            #img.set_data(env.render()) # just update the data
            #display.display(plt.gcf())
            #display.clear_output(wait=True)     
        
    
    # update agent
    mean_rewards = np.mean(rewards)
    std_rewards = np.std(rewards)

    match config["reward"]:
        case "normalized_step_reward":
            rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
        case "MC":
            rewards_ = []
            start = [sum(total_steps[:i]) for i in range(len(total_steps))]
            end =  [sum(total_steps[:i]) for i in range(1, len(total_steps)+1)]

            for s, e in zip(start, end):
                rewards_ = rewards_ + [ sum(rewards[s:e][i:])/(gamma**i) for i in range(len(rewards[s:e])) ]
            
            #rewards = rewards_
            #rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)
    

    #print(f"rewards looks like ", np.shape(rewards))  
    #print(f"log_probs looks like ", np.shape(log_probs))     
    # record training process
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # update agent
    # rewards = np.concatenate(rewards, axis=0)
    mean_rewards = np.mean(rewards)
    std_rewards = np.std(rewards)
    # rewards = (np.array(rewards) - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
    #rewards = np.array(np.sum(rewards))
    #rewards = np.array([sum(rewards[i:])/(gamma**i) for i in range(len(rewards))])
    if b % 2:
        agent.critic_learn(torch.from_numpy(np.array(rewards_)).float(), torch.stack(values).squeeze())
    else:
        agent.actor_learn(torch.stack(log_probs), torch.stack(advantages).squeeze())
    # print("logs prob looks like ", torch.stack(log_probs).size())
    # print("torch.from_numpy(rewards) looks like ", torch.from_numpy( np.array(rewards_) ).size())
    
    wandb.log({"mean rewards":mean_rewards, 
               "std reward":std_rewards, 
               "avg_total_reward": avg_total_reward, 
               "avg_final_reward": avg_final_reward, 
               "avg_episode_steps":len(rewards)/EPISODE_PER_BATCH}
    )

In [ ]:
EPISODE_PER_BATCH = 5  # update the  agent every 5 episode
NUM_BATCH = 500        # totally update the agent for 400 time

def agent_run(
    env, 
    agent, 
    config, 
    episode_per_batch=EPISODE_PER_BATCH, 
    num_batch=NUM_BATCH, 
    gamma=0.99,
    step_punish = -0.15,
    wandb_log = True,
    verbose = False
):

    if wandb_log:
        wandb.init(
            # set the wandb project where this run will be logged
            project = config['project'], 
            # track hyperparameters and run metadata
            config = config
        )

    # match config['phase']:
    #     case 'train':
    #         agent.network.train()  # Switch network into training mode 
    #     case 'eval':
    #         agent.network.eval()  # Switch network into eval mode 
    
    num_batch, episode_per_batch = config['num_batch'], config['episode_per_batch']
    avg_total_rewards, avg_final_rewards = [], []

    prg_bar = tqdm(range(num_batch))
    for batch in prg_bar:

        log_probs, rewards, values = [], [], []
        total_rewards, final_rewards = [], []
        total_steps = []
        # collect trajectory
        for episode in range(episode_per_batch):
            
            state, info = env.reset()
            total_reward, total_step = 0, 0
            terminated, truncated = False, False
            
            while not(terminated or truncated):
            # while not (terminated or truncated):
                
                agent.network.train() 
                action, log_prob, value = agent.sample(state) # at, log(at|st)
                next_state, reward, terminated, truncated, info = env.step(action)
                with torch.no_grad():
                    _, _, next_value = agent.sample(next_state)
                

                # reward += step_punish
                # if reward == -100:
                #     reward = -10
                

                log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
                state = next_state
                total_reward += reward
                
                match config["reward"]:
                    case "normalized_step_reward":
                        rewards.append(reward) # change here
                    case "MC":
                        rewards.append(reward*(gamma**total_step))
                        values.append(value)
                        advantages.append(reward + next_value - value)
                
                total_step += 1
                # ! IMPORTANT !
                # Current reward implementation: immediate reward,  given action_list : a1, a2, a3 ......
                #                                                         rewards :     r1, r2 ,r3 ......
                # medium：change "rewards" to accumulative decaying reward, given action_list : a1,                           a2,                           a3, ......
                #                                                           rewards :           r1+0.99*r2+0.99^2*r3+......, r2+0.99*r3+0.99^2*r4+...... ,  r3+0.99*r4+0.99^2*r5+ ......
                # boss : implement Actor-Critic

                # if terminated or truncated:
                #    final_rewards.append(reward)
                #    total_rewards.append(total_reward)
                #    break

            else:
                final_rewards.append(reward)
                total_rewards.append(total_reward)
                total_steps.append(total_step)
                
            

        # record training process
        avg_total_reward = sum(total_rewards) / len(total_rewards)
        avg_final_reward = sum(final_rewards) / len(final_rewards)
        avg_total_rewards.append(avg_total_reward)
        avg_final_rewards.append(avg_final_reward)
        avg_episode_steps = sum(total_steps)/len(total_steps)
        prg_bar.set_description(f"\r Total reward: {avg_total_reward: 4.1f}, Final reward: {avg_final_reward: 4.1f}")

        # update agent
        mean_rewards = np.mean(rewards)
        std_rewards = np.std(rewards)

        match config["reward"]:
            case "normalized_step_reward":
                rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # normalize the reward 
            case "MC":
                rewards_ = []
                start = [sum(total_steps[:i]) for i in range(len(total_steps))]
                end =  [sum(total_steps[:i]) for i in range(1, len(total_steps)+1)]

                for s, e in zip(start, end):
                    rewards_ = rewards_ + [ sum(rewards[s:e][i:])/(gamma**i) for i in range(len(rewards[s:e])) ]
                
                #rewards = rewards_
                #rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)
        
                #rewards = np.array([sum(rewards[i:])/(gamma**i) for i in range(len(rewards))])
                #rewards = np.array(np.sum(rewards))

        #agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
        if b % 2:
            agent.critic_learn(torch.from_numpy(np.array(rewards_)).float(), torch.stack(values).squeeze())
        else:
            agent.actor_learn(torch.stack(log_probs), torch.stack(advantages).squeeze())

        if verbose:
            print("logs prob looks like ", torch.stack(log_probs).size())
            print("torch.from_numpy(rewards) looks like ", torch.from_numpy(rewards).size())

        if wandb_log:
            wandb.log({"mean rewards":mean_rewards, 
                    "std reward":std_rewards, 
                    "avg_total_reward": avg_total_reward, 
                    "avg_final_reward": avg_final_reward, 
                    "avg_episode_steps":avg_episode_steps}
            )

    match config['phase']:
        case 'train':
            torch.save(agent.network.state_dict(), config['output_dir'])


In [ ]:
# train
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)

network = ActorCriticNetwork()
agent = ActorCriticAgent(network)

config={
    "agent": "ActorCriticAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "MC",
    "learning_rate": 1e-3,
    "num_batch": 2000,
    "episode_per_batch": 10,
    "output_dir": "./models/actor_critic_MC_gamma_0.90"
}

agent_run(env, agent, config=config, gamma=0.9)


network = ActorCriticNetwork()
agent = ActorCriticAgent(network)
agent.network.load_state_dict(torch.load(config["output_dir"]))

env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/actor_critic_MC_gamma_0.90", greedy=True)
env.reset(seed=seed)
demo_run(env, agent, seed=seed, record_video=True, video_folder = f"./video/actor_critic_MC_gamma_0.90", greedy=False)


In [ ]:
from scripts.train import trainer
from agents import (
    PolicyGradientNetwork,
    PolicyGradientAgent
) 

input_size, hidden_sizes, output_size = 8, [10, 10], 4
network = PolicyGradientNetwork(input_size, hidden_sizes, output_size)
agent = PolicyGradientAgent(network)

training_config={
    "agent": "PolicyGradientAgent",
    "phase": "train",
    "project": "lunalander",
    "reward": "discounted_future_reward",
    "learning_rate": 1e-3,
    "gamma": 0.99,
    "num_batch": 500,
    "episode_per_batch": 5,
    "model_path": "./models",
    "info": "network 8x10x10x4"
}

def discounted_future_reward(seq_rewards, rewards, gamma):
    n = len(seq_rewards)

    for i in range(2, n+1):
        seq_rewards[-i] += gamma * (seq_rewards[-i+1])
    rewards[-len(seq_rewards):] = seq_rewards
    return rewards

luna_lander_trainer = trainer(training_config, seed=543)
luna_lander_trainer.train(discounted_future_reward)